In [3]:
class PathAll(object):
    # 停用词
    stopWordList = "./stopwords/hit_stopwords.txt"
    # 词向量路径
    word2vec="./word2vec/sgns.weibo.bigram-char/sgns.weibo.bigram-char"
    # 模型路径
    model_load="./model_load/ABiLSTM_Att.weibo300.wuhan_labeled_plus.2/AdLSTM_Att_epoch_2"

    # 待预测文本
    dataSource = "./data/2class/reviews_wuhan_tencent_small.csv"
    # 预测结果的保存路径
    result_save="./data/result/result_wuhan_tencent_2class_ABiLSTM_Att_weibo300_labeled_plus_2.csv"

# 词向量的维数
Embedding_size=300
   

In [4]:
from tools import seqLength
##实例化
seqLen=seqLength(PathAll.dataSource,PathAll.stopWordList)
sequenceLen=seqLen.seq_len()

print("sequence length:",sequenceLen)
seqLen.plot_seqence()


TypeError: expected string or bytes-like object

In [45]:
from config import Config
# 实例化配置参数对象
config = Config(sequenceLen,Embedding_size)
config.rate = 0 #设置所有的语料都是测试集
config.model.dropoutKeepProb = 1.0 #全都保留下来
config.sequenceLength=87

# 读取数据集对象到内存中
import dataSet
data = dataSet.Dataset(config,PathAll)
data.dataGen()

In [46]:
print("train data shape: {}".format(data.trainReviews.shape))
print("train label shape: {}".format(data.trainLabels.shape))
print("test data shape: {}".format(data.testReviews.shape))
print("test label shape: {}".format(data.testLabels.shape))

train data shape: (0, 87)
train label shape: (0,)
test data shape: (74194, 87)
test label shape: (74194, 1)


In [47]:
from model import ABiLSTM_Attention_2class
import numpy as np
import warnings
warnings.filterwarnings("ignore")

import tensorflow.compat.v1 as tf

In [49]:
from sklearn.metrics import accuracy_score, precision_score, recall_score

predictions=[]
binaryPreds=[]

# 定义计算图
with tf.Graph().as_default():
    # 定义会话
    with tf.Session() as sess:
        lstm = ABiLSTM_Attention_2class(config, data.wordEmbedding, data.indexFreqs)
        tf.train.Saver().restore(sess, PathAll.model_load)
            
        feed_dict = {
            lstm.inputX: data.testReviews,
            lstm.inputY: data.testLabels,
            lstm.dropoutKeepProb:config.model.dropoutKeepProb
        }
        predictions, binaryPreds = sess.run(
            [lstm.predictions, lstm.binaryPreds],
            feed_dict)

        accuracy = accuracy_score(data.testLabels, binaryPreds)
        precision = precision_score(data.testLabels, binaryPreds, average='macro')
        recall = recall_score(data.testLabels, binaryPreds, average='macro')
        F = 2 * (precision * recall) / (precision + recall)
        print("acc: {}, recall: {}, F1:{}".format(accuracy,recall, F))    


INFO:tensorflow:Restoring parameters from ./model_load/ABiLSTM_Att.weibo300.wuhan_labeled_plus.2/AdLSTM_Att_epoch_2


In [ ]:
import pandas as pd
reviews = data.test_ReviewsText.tolist()
pre_sentiment = predictions
pre_class = binaryPreds
true_class = data.testLabels
correct_prediction = tf.cast(tf.equal(pre_class, true_class), tf.float32).numpy()
df1 = pd.DataFrame()
df1["reviewId"] = data._reviewId[int(len(data._reviewId)*data._rate):]
df1["reviews"] = reviews
df1["predictions"] = pre_sentiment
df1["pre_class"] = binaryPreds
df1["true_class"] = true_class
df1["accuracy"] = correct_prediction
df1.to_csv(PathAll.result_save, encoding="utf_8_sig", index=False, header=True)